# Day 13

## Part 1

With your help, the hot springs team locates an appropriate spring which launches you neatly and precisely up to the edge of Lava Island.

There's just one problem: you don't see any lava.

You do see a lot of ash and igneous rock; there are even what look like gray mountains scattered around. After a while, you make your way to a nearby cluster of mountains only to discover that the valley between them is completely full of large mirrors. Most of the mirrors seem to be aligned in a consistent way; perhaps you should head in that direction?

As you move through the valley of mirrors, you find that several of them have fallen from the large metal frames keeping them in place. The mirrors are extremely flat and shiny, and many of the fallen mirrors have lodged into the ash at strange angles. Because the terrain is all one color, it's hard to tell where it's safe to walk or where you're about to run into a mirror.

You note down the patterns of ash (.) and rocks (#) that you see as you walk (your puzzle input); perhaps by carefully analyzing these patterns, you can figure out where the mirrors are!

For example:

```
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#
```
To find the reflection in each pattern, you need to find a perfect reflection across either a horizontal line between two rows or across a vertical line between two columns.

In the first pattern, the reflection is across a vertical line between two columns; arrows on each of the two columns point at the line between the columns:

```
123456789
    ><   
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.
    ><   
123456789
```
In this pattern, the line of reflection is the vertical line between columns 5 and 6. Because the vertical line is not perfectly in the middle of the pattern, part of the pattern (column 1) has nowhere to reflect onto and can be ignored; every other column has a reflected column within the pattern and must match exactly: column 2 matches column 9, column 3 matches 8, 4 matches 7, and 5 matches 6.

The second pattern reflects across a horizontal line instead:

```
1 #...##..# 1
2 #....#..# 2
3 ..##..### 3
4v#####.##.v4
5^#####.##.^5
6 ..##..### 6
7 #....#..# 7
```
This pattern reflects across the horizontal line between rows 4 and 5. Row 1 would reflect with a hypothetical row 8, but since that's not in the pattern, row 1 doesn't need to match anything. The remaining rows match: row 2 matches row 7, row 3 matches row 6, and row 4 matches row 5.

To summarize your pattern notes, add up the number of columns to the left of each vertical line of reflection; to that, also add 100 multiplied by the number of rows above each horizontal line of reflection. In the above example, the first pattern's vertical line has 5 columns to its left and the second pattern's horizontal line has 4 rows above it, a total of 405.

Find the line of reflection in each of the patterns in your notes. What number do you get after summarizing all of your notes?

In [206]:
from IPython.display import Markdown
from typing import Optional
import pandas as pd


def find_mirror_position(df: pd.DataFrame, v: bool = False) -> Optional[int]:
    for i in range(len(df)):
        if i == 0 or i == len(df.columns) - 1:
            # can't split on the first or last row
            continue

        top_rows = df.iloc[:i]
        bottom_rows = df.iloc[i:]

        if len(top_rows) > len(bottom_rows):
            # shorten top rows
            top_rows = top_rows.tail(len(bottom_rows))
        elif len(top_rows) < len(bottom_rows):
            # shorten bottom rows
            bottom_rows = bottom_rows.head(len(top_rows))

        # flip the bottom rows' index positions so they (potentially) match the top
        bottom_rows = bottom_rows.iloc[::-1]

        assert len(bottom_rows) == len(top_rows)

        t = top_rows.to_string(index=False, header=False)
        # print(t)
        b = bottom_rows.to_string(index=False, header=False)
        # print(b)
        # print(f"{t == b}")
        if t == b:
            # display(pd.concat([top_rows, bottom_rows.iloc[::-1]], ignore_index=True))
            debug_df = pd.concat([top_rows, bottom_rows.iloc[::-1]], ignore_index=True)
            if v:
                debug_df = debug_df.transpose()
                df = df.transpose()

            # last row's index + 1 in `top_rows` is the position of the mirror line
            last_index = top_rows.index[-1]
            position = len(df.iloc[: last_index + 1])
            # 1x for mirroring against a vertical line, 100x for a horizontal line
            score = position * 100 if not v else position
            display(
                Markdown(
                    f"### line between {position-1}-{position} {'vertical' if v else 'horizontal'} -> {score=}"
                )
            )
            print(f"{len(top_rows)=} -> {last_index=}")

            def stuff(s, b: int):
                return [
                    "border-bottom: 3px dashed gold; color:gold" if s.name <= b else "" for v in s
                ]

            display(
                df.style.apply(
                    stuff,
                    b=last_index,
                    axis=int(not v),
                )
                .map_index(
                    lambda x: "background-color: #99000077"
                    if x in top_rows.index
                    else "background-color: #00009977"
                    if x in bottom_rows.index
                    else "",
                    axis=int(v),
                )
                .map(lambda x: "background-color: #00cccc77" if x == 1 else "")
                .map_index(
                    lambda x: "color:gold" if x <= max(top_rows.index) else None,
                    axis=int(v),
                )
            )
            return score


def find_mirror_lines(data: str):
    rows = data.split("\n")
    df = pd.DataFrame([list(line) for line in rows]).astype(int)

    score = 0

    horizontal_lines = find_mirror_position(df)
    if horizontal_lines is not None:
        score += horizontal_lines

    vertical_lines = find_mirror_position(df.transpose(), v=True)
    if vertical_lines is not None:
        score += vertical_lines

    return score


test_input = (
    """#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#""".replace(
        "#", "1"
    )
    .replace(".", "0")
    .split("\n\n")
)

test_total_score = 0
for i, test_block in enumerate(test_input):
    display(Markdown(f"# `{i}`"))
    test_score = find_mirror_lines(test_block)
    test_total_score += test_score
test_total_score

# `0`

### line between 4-5 vertical -> score=5

len(top_rows)=4 -> last_index=4


,0,1,2,3,4,5,6,7,8
0,1,0,1,1,0,0,1,1,0
1,0,0,1,0,1,1,0,1,0
2,1,1,0,0,0,0,0,0,1
3,1,1,0,0,0,0,0,0,1
4,0,0,1,0,1,1,0,1,0
5,0,0,1,1,0,0,1,1,0
6,1,0,1,0,1,1,0,1,0


# `1`

### line between 3-4 horizontal -> score=400

len(top_rows)=3 -> last_index=3


,0,1,2,3,4,5,6,7,8
0,1,0,0,0,1,1,0,0,1
1,1,0,0,0,0,1,0,0,1
2,0,0,1,1,0,0,1,1,1
3,1,1,1,1,1,0,1,1,0
4,1,1,1,1,1,0,1,1,0
5,0,0,1,1,0,0,1,1,1
6,1,0,0,0,0,1,0,0,1


405

In [207]:
inputs = open("../inputs/13.txt").read().replace("#", "1").replace(".", "0").split("\n\n")
score = 0
for block in inputs:
    score += find_mirror_lines(block)

### line between 11-12 vertical -> score=12

len(top_rows)=1 -> last_index=11


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,1,1,0,1,1,0,1,1,0,0,0,0
1,1,0,1,1,1,1,1,1,0,1,1,0,0
2,1,1,0,0,0,0,0,0,1,1,1,0,0
3,0,0,0,1,1,1,1,0,0,0,0,0,0
4,0,0,0,1,1,1,1,0,0,0,1,1,1
5,0,0,1,0,0,0,0,1,0,0,1,0,0
6,0,1,0,1,1,1,1,0,1,0,1,1,1
7,0,0,1,0,0,0,0,1,0,0,1,1,1
8,0,1,1,0,1,1,0,1,1,0,0,0,0
9,0,0,1,1,1,1,1,1,0,0,1,1,1


### line between 7-8 horizontal -> score=800

len(top_rows)=1 -> last_index=7


...

len(top_rows)=1 -> last_index=9


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,1,1,1,1,1,1,1,0,1,1,0,1
1,1,0,1,1,1,1,0,1,1,1,1,1,0
2,1,1,1,1,1,1,1,1,1,1,0,1,1
3,1,1,1,1,1,1,0,1,0,1,1,1,0
4,0,0,1,0,0,1,0,0,1,1,0,0,0
5,1,0,1,0,0,1,0,1,0,0,1,1,1
6,0,0,1,0,0,1,0,0,0,1,1,1,0
7,0,0,0,1,1,0,0,0,1,1,1,0,0
8,1,0,0,1,1,0,0,1,1,0,0,0,1
9,1,1,0,1,1,0,1,1,1,0,1,1,0


### line between 0-1 horizontal -> score=100

len(top_rows)=1 -> last_index=0


,0,1,2,3,4,5,6,7,8,9,10
0,1,1,1,0,1,1,1,1,1,0,1
1,1,1,1,0,1,1,1,1,1,0,1
2,1,0,0,1,1,0,1,1,1,1,0
3,1,1,1,1,0,1,0,0,0,1,0
4,1,0,0,0,0,0,1,0,0,1,1
5,0,1,1,0,1,0,0,1,1,0,1
6,1,1,0,1,1,0,1,0,0,0,1
7,1,1,0,1,1,0,1,0,0,0,1
8,0,1,0,0,1,0,0,1,1,0,1
9,1,0,0,0,0,0,1,0,0,1,1


In [208]:
score

23191

14000 too low
23191 too low
23204 nope
23234 too high